In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_absolute_error

In [2]:
turb_1 = pd.read_csv('../data/FRSSO18.csv',sep=';',decimal=',',parse_dates=['DateTime'],dayfirst=True)

In [3]:
turb_1.head(3)

,DateTime,ActivePower,ProductionForecast_D,ProductionForecast_M
0,2020-03-05 00:00:00,1614.345422,948.189357,745.2
1,2020-03-05 01:00:00,1478.355419,967.269962,711.8
2,2020-03-05 02:00:00,1690.147069,948.830677,645.0


In [4]:
turb_1 = turb_1[turb_1.ActivePower != 0]
turb_1 = turb_1[turb_1.ProductionForecast_D != 0]
turb_1 = turb_1[turb_1.ProductionForecast_M != 0]

In [5]:
turb_1['error_expert1'] = abs(turb_1['ActivePower'] - turb_1['ProductionForecast_D'])/turb_1['ActivePower']

In [6]:
turb_1['error_expert2'] = abs(turb_1['ActivePower'] - turb_1['ProductionForecast_M'])/turb_1['ActivePower']

In [7]:
turb_1['min_error'] = turb_1[['error_expert1','error_expert2']].min(axis=1)

In [8]:
turb_1.loc[turb_1.index[0], 'weight_expert1'] = 1 / (1 + (turb_1.loc[turb_1.index[0], 'error_expert1']/turb_1.loc[turb_1.index[0], 'error_expert2']))
turb_1.loc[turb_1.index[0], 'weight_expert2'] = 1 - turb_1.loc[turb_1.index[0], 'weight_expert1']

In [9]:
for k in range(1, len(turb_1)):
    turb_1.loc[turb_1.index[k], 'weight_expert1'] = (
        turb_1.loc[turb_1.index[k-1], 'weight_expert1'] + 1 /
            (1 + (turb_1.loc[turb_1.index[k], 'error_expert1'] /
             turb_1.loc[turb_1.index[k], 'error_expert2'])))
    turb_1.loc[turb_1.index[k], 'weight_expert2']=(
        turb_1.loc[turb_1.index[k-1], 'weight_expert1'] + 1 /
            (1 + (turb_1.loc[turb_1.index[k], 'error_expert2'] /
             turb_1.loc[turb_1.index[k], 'error_expert1'])))
    
turb_1['sum_weights']=turb_1['weight_expert1'] + turb_1['weight_expert2']

In [10]:
turb_1['new_expert'] = (turb_1['ProductionForecast_D']*turb_1['weight_expert1'] +
                        turb_1['ProductionForecast_M']*turb_1['weight_expert2'])/turb_1['sum_weights']

In [12]:
ProductionForecast_D = turb_1['ProductionForecast_D'].to_numpy()
ProductionForecast_M = turb_1['ProductionForecast_M'].to_numpy()
New_expert = turb_1['new_expert'].to_numpy()
Y_true = turb_1['ActivePower'].to_numpy()

In [23]:
mae_D = mean_absolute_error(Y_true, ProductionForecast_D)
mae_M = mean_absolute_error(Y_true, ProductionForecast_M)
mae_New = mean_absolute_error(Y_true, New_expert)

In [24]:
d = {'mae_D': [mae_D], 'mae_M': [mae_M], 'mae_New': [mae_New]}
df = pd.DataFrame(data=d)
df

,mae_D,mae_M,mae_New
0,245.464623,237.088207,227.282785
